In [1]:
import pandas as pd
from PIL import Image
import pytesseract
import sys
from pdf2image import convert_from_path
import os
from pytesseract import image_to_string

In [2]:
path = os.getcwd()
path

"/Users/ellenyz/OneDrive - UW-Madison/DrGreen'sLab/Medicare Certificate/1974"

In [3]:
#f_list = os.listdir(path)
f_list = ['AL_ID_cropped.pdf',
          'IL_MI_cropped.pdf',
          'MN_NM_cropped.pdf',
          'NY_RI_cropped.pdf',
          'SC_WY_cropped.pdf']

#Convert pdf to images
image_counter = 1
for f in f_list:
    PDF_file = path + '/' + f
    pages = convert_from_path(PDF_file, 500)
    for page in pages:
        # Declaring filename for each page of PDF as JPG
        # For each page, filename will be:
        # PDF page 1 -> page_1.jpg
        # PDF page 2 -> page_2.jpg
        # PDF page 3 -> page_3.jpg
        # ....
        # PDF page n -> page_n.jpg
        filename = "page_"+str(image_counter)+".jpg"
      
        # Save the image of the page in system
        page.save(path+ '/' + filename, 'JPEG')
  
        # Increment the counter to update filename
        image_counter = image_counter + 1

In [4]:
out = pd.DataFrame(columns=['HOSP','STATE'])

In [5]:
def my_function(state,a,b):
    outfile = path + '/' +'out_text_'+state+'.txt'
    f = open(outfile, "a")
    
    for i in range(a, b):
        filename = "page_"+str(i)+".jpg"
        # Recognize the text as string in image using pytesserct
        text = str(((pytesseract.image_to_string(Image.open(path+'/'+filename)))))
        text = text.replace('-\n', '')    
        # Finally, write the processed text to the file.
        
        f.write(text)
    
    f.close()
    with open(outfile) as f:
        line = f.read().splitlines() 
    df = pd.DataFrame(line)
    df.columns =['HOSP']
    data = df[df['HOSP'].str.contains('Hosp'or'Med Center'or'Medical Center')]
    data.reset_index(inplace = True)
    data = pd.DataFrame(data.HOSP)
    data['STATE'] = state
    return(data)

In [6]:
ref = pd.read_excel('reference.xlsx',header=None,names=['state','p1','p2'])

In [7]:
for i in range(0,51):
    state = ref.iloc[i,0]
    a = ref.iloc[i,1]
    b = ref.iloc[i,2]
    out = out.append(my_function(state,a,b))

In [8]:
zipcode = out['HOSP'].str.findall(r'(\d{5})')

In [9]:
zipcode = pd.DataFrame(zipcode)

In [10]:
zipcode = zipcode.rename(columns={"HOSP": "ZIP"})
zipcode.reset_index(inplace = True)
zipcode = pd.DataFrame(zipcode.ZIP)
zipcode

,ZIP
0,[]
1,[36310]
2,[35007]
3,[35010]
4,[35442]
...,...
6791,[25271]
6792,[25276]
6793,[26651]
6794,[26003]


In [11]:
writer = pd.ExcelWriter('1974.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet. you could write different string like above if you want
out.HOSP.to_excel(writer, sheet_name='HOSP_name + ad')
out.STATE.to_excel(writer, sheet_name='City_State')
zipcode.to_excel(writer, sheet_name='Zip_Code')

# Close the Pandas Excel writer and output the Excel file.
writer.save()